#### Test mynds packages in the sandbox

In [1]:
from functools import partial
from pathlib import Path

import cv2
import onnxruntime as ort
import tqdm

from mynd.camera import CameraID
from mynd.image import Image, filter_image_clahe
from mynd.io import read_image
from mynd.visualization import KeyCode, destroy_all_windows


from mynd.visualization.stereo_visualizer import (
    StereoVisualizer,
    ImageWindow,
    ImageWindowsManager,
)


from mynd.utils.filesystem import list_directory
from mynd.utils.log import logger
from mynd.utils.result import Ok, Err, Result


logger.info(f"Device: {ort.get_device()}")


def main() -> None:
    """Main entrypoint."""

    DIRECTORY: Path = Path(
        "/data/kingston_snv_01/acfr_images_grayworld/r23m7ms0_20140616_044549_grayworld"
    )
    PATTERN: str = "*.png"

    visualizer: StereoVisualizer = StereoVisualizer(
        stereo_methods=dict(), source=list()
    )
    window: ImageWindow = ImageWindow("my_test_window", size=(1000, 500))

    logger.info(visualizer)

    raise NotImplementedError


# INVOKE MAIN
main()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/home/martin/.cache/pypoetry/virtualenvs/mynd-qzh2EXyG-py3.11/lib/python3.11/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
2024-10-21 09:40:23.526 | INFO     | __main__:<module>:22 - Device: GPU


TypeError: object() takes no arguments